In [1]:
"""
pull data in the 10-K reports around the word 'risk'
ran for one year at a time 2011-2020
"""

"\npull data in the 10-K reports around the word 'risk'\n"

In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict


from bs4 import BeautifulSoup
import re
import requests
import unicodedata
import random
import time

In [2]:
df = pd.read_pickle("../data/urls_2013.pickle")

df.head()

,id,accessionNo,ticker,cik,companyName,companyNameLong,formType,filedAt,linkToTxt,linkToHtml,irsNo,fiscalYearEnd,sic,filed_at
0,70510156e6c3b638104857d5b259ba5b,0000063754-13-000002,MKC,63754,MCCORMICK & CO INC,MCCORMICK & CO INC (Filer),10-K,2013-01-25T08:41:19-05:00,https://www.sec.gov/Archives/edgar/data/63754/...,https://www.sec.gov/Archives/edgar/data/63754/...,520408290,1130,2090 Miscellaneous Food Preparations &amp; Kin...,2013-01-25 08:41:19-05:00
1,8dc96e31f7b822b065dbcfb1b3e9ba61,0000063908-13-000010,MCD,63908,MCDONALDS CORP,MCDONALDS CORP (Filer),10-K,2013-02-25T11:19:42-05:00,https://www.sec.gov/Archives/edgar/data/63908/...,https://www.sec.gov/Archives/edgar/data/63908/...,362361282,1231,5812 Retail-Eating Places,2013-02-25 11:19:42-05:00
2,08b85e33e0818b2ec63af72f9a5c7498,0001561787-13-000010,MCK,927653,MCKESSON CORP,MCKESSON CORP (Filer),10-K,2013-05-07T16:33:17-04:00,https://www.sec.gov/Archives/edgar/data/927653...,https://www.sec.gov/Archives/edgar/data/927653...,943207296,0331,"5122 Wholesale-Drugs, Proprietaries &amp; Drug...",2013-05-07 16:33:17-04:00
3,c0c84c82d0fb6910f12e23cfe1d87111,0001193125-13-084618,MRK,310158,Merck & Co. Inc.,Merck & Co. Inc. (Filer),10-K,2013-02-28T16:54:26-05:00,https://www.sec.gov/Archives/edgar/data/310158...,https://www.sec.gov/Archives/edgar/data/310158...,221918501,1231,2834 Pharmaceutical Preparations,2013-02-28 16:54:26-05:00
4,eb7358c45f377fb83fcf34d6a714712d,0001193125-13-077792,MET,1099219,METLIFE INC,METLIFE INC (Filer),10-K,2013-02-26T21:13:26-05:00,https://www.sec.gov/Archives/edgar/data/109921...,https://www.sec.gov/Archives/edgar/data/109921...,134075851,1231,6311 Life Insurance,2013-02-26 21:13:26-05:00


In [6]:
#create dict for webscrape below
form_dict = defaultdict(list)
for inx, row in df.iterrows():
    form_dict[row['ticker']].append(row['linkToTxt'])


In [10]:
text = {}

for ticker, urls in form_dict.items():
    for url in urls:
        res = requests.get(url)
        page = res.content
        
        try: 
            soup = BeautifulSoup(page, 'html.parser')
        except:
            text[ticker] = "error"
            continue

        # some documents use HTML div tags and others paragraph tags
        paragraphs = [paragraph for paragraph in soup.find_all('p')]
        divs = [div for div in soup.find_all('div')]

        blobs = []

        # search for "risk" or "risks"
        for paragraph in paragraphs:
            if paragraph.find_all(text=re.compile(r"(\brisk\b|\brisks\b)", re.IGNORECASE)): 
                for element in paragraph.find_all(text=re.compile(r"(\brisk\b|\brisks\b)", re.IGNORECASE)):
                    blobs.append(str(unicodedata.normalize('NFKD', element)))

        for div in divs:
            if div.find_all(text=re.compile(r"(\brisk\b|\brisks\b)", re.IGNORECASE)): 
                for element in div.find_all(text=re.compile(r"(\brisk\b|\brisks\b)", re.IGNORECASE)):
                    blobs.append(str(unicodedata.normalize('NFKD', element)))

        # remove any unparsed HTML 
        for blob in blobs: 
            if "font-family" in blob:
                blobs.remove(blob)

        text[ticker + ";" + url] = blobs 

        # pause before querying next document
        time.sleep(.5+2*random.random()) 
     
    
print(len(text))

with open('text_risk_data_2013.pickle', 'wb') as to_write:
    pickle.dump(text, to_write)

/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![6^XM;]+^4J\nM>)9U@'
  warnings.warn(msg)
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![\nM(6K%;D`9XE>MC&7'
  warnings.warn(msg)
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'BC' in marked section
  warnings.warn(msg)
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'WT.' in marked section
  warnings.warn(msg)
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'Q7P' in marked section
  warnings.warn(msg)
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at "<![`XJ\nM2'0K

466
